# Rotton Tomatoes Movie Review Sentiment Analysis
### How useful is TF-IDF weighing in sentiment analysis?

This kernel works with data from the Movie Review Sentiment Analysis Competition on Kaggle.com. More info can be found at https://nlp.stanford.edu/sentiment/.

In the report created by the Stanford research group, each sentence is seperated into it's component pieces and given a positive/negative scoring. This is summed up as a word tree to give each sentence an overall sentiment.
<img src="Capture.png">
*Image taken from nlp.stanford.edu*

The objective of this kernel is to explore using tf-idf weighting factors to assist traditional classifier models. Then we will compare these models to simple 'bag of words' text analysis options. Is TF-IDF underutilized in sentiment analysis?

## Step 1: Reading SA data

In [ ]:
import csv
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn import metrics
from sklearn.model_selection import train_test_split   
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

In [ ]:
train = pd.read_csv('../input/train.tsv', sep='\t')
test = pd.read_csv('../input/test.tsv',  sep='\t')
sampleSub = pd.read_csv('../input/sampleSubmission.csv')

## Step 2: Analysing data

### Dataframe analysis

In [ ]:
print(train.shape, "\n", 
      test.shape
     )

In [ ]:
print (train.isnull().values.any(), "\n",
      test.isnull().values.any()
      )

In [ ]:
train.head()

In [ ]:
test.head()

Number of unique sentences in the training / testing dataset

In [ ]:
len(train.groupby('SentenceId').nunique())

In [ ]:
len(test.groupby('SentenceId').nunique())

Let's create a dataset with only full sentences. Exploring data this way will gives us cleaner graphs that aren't biased toward longer sentences. We can also add a label for the sentiment value to increased readability.

In [ ]:
#Create df of full sentences
fullSent = train.loc[train.groupby('SentenceId')['PhraseId'].idxmin()]

#Change sentiment to increase readability
fullSent['sentiment_label'] = ''
Sentiment_Label = ['Negative', 'Somewhat Negative', 
                  'Neutral', 'Somewhat Positive', 'Positive']
for sent, label in enumerate(Sentiment_Label):
    fullSent.loc[train.Sentiment == sent, 'sentiment_label'] = label
    
fullSent.head()

## NLP Analysis

In natural language processing, the gold standard of text analysis is TF-IDF analysis. TF-IDF is short for "term frequency–inverse document frequency", and it focus on finding the 'valuable' classification words for each document. Term frequency of a word in a document increases it's score, and document frequency (the number of documents where the word shows up) decreases the score.

Scikit-learn provides a TfidfVectorizer class, which has the ability to remove common stop words (like a, the, and, or). I also created a CountVectorizer class object for comparison in the evaluation section below. Below we'll try to find the most 'valuable words' for each sentiment group. 

In [ ]:
#Add non-helpful stopwords to stopword list
Stopwords = list(ENGLISH_STOP_WORDS)
Stopwords.extend(['movie','movies','film','nt','rrb','lrb',
                      'make','work','like','story','time','little'])

#Create tfidf vectorizer object & fit to full sentence training data
tfidf_vectorizor = TfidfVectorizer(min_df=5, 
                             max_df=0.5,
                             analyzer='word',
                             strip_accents='unicode',
                             ngram_range=(1, 3),
                             sublinear_tf=True, 
                             smooth_idf=True,
                             use_idf=True,
                             stop_words=Stopwords)

tfidf_vectorizor.fit(list(fullSent['Phrase']))


#Create bag of word vectorizer for comparison in evaluation section
BoW_vectorizer = CountVectorizer(strip_accents='unicode',
                                 stop_words=Stopwords,
                                 ngram_range=(1,3),
                                 analyzer='word',
                                 min_df=5,
                                 max_df=0.5)

BoW_vectorizer.fit(list(fullSent['Phrase']))

In [ ]:
#functions to create graphics below from tf-idf matrices
#adapted from : https://buhrmann.github.io/tfidf-analysis.html
def top_tfidf_feats(row, features, top_n=20):
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

def top_feats_in_doc(Xtr, features, row_id, top_n=20):
    row = np.squeeze(Xtr[row_id].toarray())
    return top_tfidf_feats(row, features, top_n)

def top_mean_feats(Xtr, features, grp_ids=None, min_tfidf=0.1, top_n=10):
    if grp_ids:
        D = Xtr[grp_ids].toarray()
    else:
        D = Xtr.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    return top_tfidf_feats(tfidf_means, features, top_n)

def top_feats_by_class(Xtr, y, features, min_tfidf=0.1, top_n=16):
    dfs = []
    labels = np.unique(y)
    for label in labels:
        ids = np.where(y==label)
        feats_df = top_mean_feats(Xtr, features, ids, min_tfidf=min_tfidf, top_n=top_n)
        feats_df.label = label
        dfs.append(feats_df)
    return dfs

def plot_tfidf_classfeats_h(dfs, num_class=9):
    fig = plt.figure(figsize=(12, 100), facecolor="w")
    x = np.arange(len(dfs[0]))
    for i, df in enumerate(dfs):
        ax = fig.add_subplot(num_class, 1, i+1)
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.set_frame_on(False)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()
        ax.set_xlabel("Mean Tf-Idf Score", labelpad=16, fontsize=16)
        ax.set_ylabel("Word", labelpad=16, fontsize=16)
        ax.set_title(str(df.label) + ' Sentiment Class', fontsize=25)
        ax.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
        ax.barh(x, df.tfidf, align='center')
        ax.set_yticks(x)
        ax.set_ylim([-1, x[-1]+1])
        ax.invert_yaxis()
        yticks = ax.set_yticklabels(df.feature)
        
        for tick in ax.yaxis.get_major_ticks():
                tick.label.set_fontsize(20) 
        plt.subplots_adjust(bottom=0.09, right=0.97, left=0.15, top=0.95, wspace=0.52)
    plt.show()


The graphs below shows the words with the highest tf-idf score in each class. The matplotlib code is adapted from https://buhrmann.github.io/tfidf-analysis.html, a great analysis of what tf-idf scores can show you about a set of documents in a corpus. Note we use the full sentence dataset created above.

In [ ]:
class_Xtr = tfidf_vectorizor.transform(fullSent['Phrase'])
class_y = fullSent['sentiment_label']
class_features = tfidf_vectorizor.get_feature_names()
class_top_dfs = top_feats_by_class(class_Xtr, class_y, class_features)
plot_tfidf_classfeats_h(class_top_dfs, 7)

## Part 3: Sentiment Analysis using Machine Learning Models

### Train/Test Splitting

In [ ]:
phrase = np.array(train['Phrase'])
sentiment = np.array(train['Sentiment'])
# build train and test datasets
phrase_train, phrase_test, sentiment_train, sentiment_test = train_test_split(phrase, 
                                                                              sentiment, 
                                                                              test_size=0.2, 
                                                                              random_state=4)

#TF-IDF
train_tfidfmatrix = tfidf_vectorizor.fit_transform(phrase_train)
test_tfidfmatrix = tfidf_vectorizor.transform(phrase_test)

#Vectorizer (Bag of Words Model)
train_simplevector = BoW_vectorizer.transform(phrase_train)
test_simplevector = BoW_vectorizer.transform(phrase_test)

In [ ]:
def train_model_predict (classifier, train_features, train_labels,
                      test_features):
    classifier.fit(train_features, train_labels)
    predictions = classifier.predict(test_features)
    return predictions

### Naive Bayes Model

#### TF-IDF vectorization

In [ ]:
model1 = MultinomialNB() 
NBPredictions = train_model_predict(model1, train_tfidfmatrix, sentiment_train,
                             test_tfidfmatrix)

#### 'Bag of Words' vectorization

In [ ]:
NBPredictions2 = train_model_predict(model1, train_simplevector, sentiment_train,
                             test_simplevector)

### Logistic Regression Model

#### TF-IDF vectorization

In [ ]:
model2 = LogisticRegression(solver = 'liblinear', multi_class = 'ovr')
LogisticRegressionPredictions = train_model_predict(model2, train_tfidfmatrix, sentiment_train,
                             test_tfidfmatrix)

#### 'Bag of Words' vectorization

In [ ]:
LogisticRegressionPredictions2 = train_model_predict(model2, train_simplevector, sentiment_train,
                             test_simplevector)

## Part 4 : Evaluation

Since we don't care about certain types of scoring over others (ie weighing mislabeling more than correctly labeling), we'll use F1 scores as shorthand for how good the model is.

In [ ]:
def get_metrics(true_labels, predicted_labels, feature):  
    print(feature)
    print('Accuracy:', np.round(metrics.accuracy_score(true_labels, 
                                               predicted_labels), 4))
    print('Precision:', np.round(metrics.precision_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'), 4))
    print('Recall:', np.round(metrics.recall_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'), 4))
    print('F1 Score:', np.round(metrics.f1_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'), 4))
    print('\n')
    

In [ ]:
get_metrics(NBPredictions, sentiment_test, 'Naive Bayes & TF-IDF Scores: ')
get_metrics(NBPredictions2, sentiment_test, 'Naive Bayes & Bag of Words Scores: ')

In [ ]:
get_metrics(LogisticRegressionPredictions, sentiment_test, 'Logistic Regression & TF-IDF Scores: ')
get_metrics(LogisticRegressionPredictions2, sentiment_test, 'Logistic Regression & Bag of Words Scores: ')

### Table of F1 scores of different modelling methods

|            |NaiveBayes|LogReg|
|------------|----------|------|
|   TF-IDF   | 0.6577   |0.6615|
|Bag of Words| 0.6386   |0.6485|

## Part 5: Conclusion

TF-IDF weighing does have a positive effect on the correctness of the model. The difference between a bag-of-words weighing schema and TF-IDF schema is slight but consistent between both classifiers.

We could also run a GridSearch to find the most correct parameters for each model, though we run the risk of overfitting for the Kaggle test data. Future testing can be done on different models and more or less optimizeed ones - including nerual networks.

References and sources used:

https://buhrmann.github.io/tfidf-analysis.html
http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction
https://www.dataquest.io/blog/naive-bayes-tutorial/

#### Application to competition test set

In [ ]:
train_tfidf = tfidf_vectorizor.fit_transform(train['Phrase'])
model2.fit(train_tfidf, train['Sentiment'])
test_tfidf = tfidf_vectorizor.transform(test['Phrase'])
predictions = model2.predict(test_tfidf)

test['Sentiment'] = predictions
submission = test[['PhraseId','Sentiment']]
submission.to_csv('submission.csv',index=False)